In [1]:
import pandas as pd
import folium
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
import matplotlib.colors as colors

In [2]:
data = pd.read_csv('example_company_data.csv')
data.head()

,lot_id,company_name,type,make,model,quantity,address,progress
0,1,Athena Hackathon,laptop,Apple,MacBook Pro (2018),2,"10 South Pl, London EC2M 7EB",Requested
1,2,Pwc,mobile phone,Huaweii,p30 pro,7,"1 Embankment Pl, London WC2N 6RH",Allocated
2,3,Beamery,tablet,Apple,iPad mini 2019,3,"97-101 Hackney Rd, London E2 8ET",Available
3,4,Visa,laptop,Acer,Swift 3 AMD,4,"107 Fleet St, London EC4A 2AB",Collected
4,5,Exclaimer,mobile phone,Lenovo,ThinkPad E14,12,"250 Fowler Ave, Farnborough GU14 7JP",Available


In [3]:
geolocator = Nominatim(user_agent="myGeocoder")
location = geolocator.geocode("10 South Pl, London, EC2M 7EB")

In [4]:
print(location)

South Place, Cornhill, City of London, England, EC2M 2AF, United Kingdom


In [5]:
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 51.5188151, Longitude = -0.0863075


In [6]:
geolocator = Nominatim(user_agent="myGeocoder")
def latLonFinder(address):
    try: 
        location = geolocator.geocode(str(address))
        return pd.Series([location.latitude, location.longitude])
    except:
        return pd.Series([0,0])

data[["lat","lon"]] = data['address'].apply(latLonFinder)

data.head()

,lot_id,company_name,type,make,model,quantity,address,progress,lat,lon
0,1,Athena Hackathon,laptop,Apple,MacBook Pro (2018),2,"10 South Pl, London EC2M 7EB",Requested,51.518815,-0.086307
1,2,Pwc,mobile phone,Huaweii,p30 pro,7,"1 Embankment Pl, London WC2N 6RH",Allocated,51.507818,-0.123686
2,3,Beamery,tablet,Apple,iPad mini 2019,3,"97-101 Hackney Rd, London E2 8ET",Available,51.527758,-0.076484
3,4,Visa,laptop,Acer,Swift 3 AMD,4,"107 Fleet St, London EC4A 2AB",Collected,51.514299,-0.104835
4,5,Exclaimer,mobile phone,Lenovo,ThinkPad E14,12,"250 Fowler Ave, Farnborough GU14 7JP",Available,51.285018,-0.759914


In [9]:
m = folium.Map(
    location=[51.509865,-0.118092],
    tiles="cartodbpositron",
    zoom_start=11)

laptop = folium.FeatureGroup("Laptops")
tablet = folium.FeatureGroup("Tablets")
mobile_phone = folium.FeatureGroup("Mobile Phones")
available = data[(data['progress']=='Requested') | (data['progress']=='Available')]

for company_name, cat, make, model, quantity, lat, lon in zip(available['company_name'], available['type'], available['make'],available['model'],available['quantity'],available['lat'],available['lon']):
    colors_dict = dict([("laptop","#fab30c"),("mobile phone","#2cb0f2"),("tablet","#fa0c8b")])
    group_dict = dict([("laptop",laptop),("mobile phone",mobile_phone),("tablet",tablet)])
    color = colors_dict.get(cat)
    popup_text="""{}<br> Category: {}<br> {} {}<br> Quantity: {}"""
    popup_text = popup_text.format(company_name, cat, make, model, quantity)
    label = folium.Popup(popup_text)
    folium.CircleMarker([lat, lon], radius=10, fill=True, popup = label, fill_color=color, color=color).add_to(group_dict.get(cat))
    
laptop.add_to(m)
tablet.add_to(m)
mobile_phone.add_to(m)

folium.LayerControl().add_to(m)
m

In [10]:
m.save("map.html")